In [1]:
# add de folder to path
import sys
sys.path.insert(1, '../models/')
sys.path.insert(1, '../de/')

from time import time
import numpy as np
from scipy.linalg import expm
from qiskit.quantum_info.operators import Operator
from models import BaseModel
from signals import Constant, ConstantSignal, PiecewiseConstant
from DE_Problems import StateVectorProblem, UnitaryProblem
from DE_Options import DE_Options
from Solver import solve, Solver

X = np.array([[0., 1.], [1., 0.]], dtype=complex)
Y = np.array([[0., -1j], [1j, 0.]], dtype=complex)
Z = np.array([[1., 0.], [0., -1.]], dtype=complex)
Sp = np.array([[0., 0.], [1., 0.]])
Sm = np.array([[0., 1.], [0., 0.]])

op_X = Operator.from_label('X')
op_Y = Operator.from_label('Y')
op_Z = Operator.from_label('Z')

## 1. DE Methods

A standardized interface to solving routines, broken into two types:
- `ODE_Method`s, which are standard 'ODE' solving routines using the rhs function
- `BMDE_Method`s, which are 'BMDE' solving routines that can use the generator

In [2]:
from DE_Methods import Expm, ScipyODE

### 1.1 ScipyODE

An interface to the `solve_ivp` function in scipy. This is a subclass of `ODE_Method`

In [22]:
t0 = 0
y0 = np.array([[1., 0.], [0., 1.]], dtype=complex)
def rhs(t, y):
    if t < 1.:
        return (-1j * (np.pi/2)* X ) @ y
    else:
        return (-1j * (np.pi/2)* Y ) @ y

# 'method' key should have a value acceptable by solve_ivp
# 'scipy_options' key is for options that are passed to solve_ivp
#solver_options = {'method': 'RK45', 'scipy_options' : {'atol': 10**-9, 'rtol': 10**-9}}
solver_options = DE_Options(method='RK45', atol=10**-8, rtol=10**-8)

scipy_solver = ScipyODE(t0, y0, rhs=rhs, options=solver_options)

In [23]:
scipy_solver.integrate(2.)
scipy_solver.y

array([[ 3.88913686e-13+9.99999988e-01j,  3.17904482e-07-3.15573643e-07j],
       [-3.17904482e-07-3.15573643e-07j,  3.88913686e-13-9.99999988e-01j]])

In [24]:
expm(-1j*(np.pi/2)*Y) @ expm(-1j*(np.pi/2)*X)

array([[0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j]])

## 1.2 Expm

This is a `BMDE_Method` based on matrix exponentiation of the generator.

In [27]:
# initial state
t0 = 0.
y0 = np.array([[1., 0], [0., 1.]])

# define generator
def generator(t):
    if t < 1.:
        return -1j*(np.pi/2)*X
    else:
        return -1j*(np.pi/2)*Y

options = DE_Options(max_dt=0.01)

expm_solver = Expm(t0, y0, rhs=generator, options=options)

In [28]:
expm_solver.integrate(2.)
expm_solver.y

array([[5.19108852e-29+1.00000000e+00j, 7.06726344e-15-7.55799043e-15j],
       [7.33441086e-15+7.19782338e-15j, 5.01495294e-29-1.00000000e+00j]])

## 2. Models

This is a rudimentary model - a list of operators and signals. These are defined using terra operators.

Here we define a qubit hamiltonian using the model object.

In [29]:
w = 5.
r = 0.2

operators = [2 * np.pi * w * op_Z / 2, 2 * np.pi * r * op_X / 2]
signals = [Constant(1.), 
           ConstantSignal(1., carrier_freq=w)]

qubit_ham_model = BaseModel(signals, operators)

We can evaluate the model at a given time (i.e. take the linear combination at the given time).

In [30]:
qubit_ham_model.evaluate(2.)

array([[ 15.70796327+0.j,   0.62831853+0.j],
       [  0.62831853+0.j, -15.70796327+0.j]])

We can apply the model at a given time to a given vector.

In [31]:
qubit_ham_model.lmult(2., np.array([1., 0.]))

array([15.70796327+0.j,  0.62831853+0.j])

## 3. BMDE problems and solvers

`BMDE_Problem` class represents a DE, including the generator model, the initial state, and the initial time. We subclass this to set up different types of BMDEs.

## 3.1 Schrodinger statevector problem

In [32]:
t0 = 0.
y0 = np.array([1., 0.], dtype=complex)

# takes the hamiltonian and sets up Schrodinger equation for the statevector
state_problem = StateVectorProblem(hamiltonian=qubit_ham_model,
                                   y0=y0,
                                   t0=t0)

solver_options = DE_Options(method='scipy-RK45', atol=10**-8, rtol=10**-8)

scipy_solver = Solver(state_problem, options=solver_options)

In [33]:
scipy_solver.integrate(1./r) # do a pi pulse
scipy_solver.y

array([-1.96352698e-05-0.00500054j,  2.82321676e-10+0.99998751j])

### 3.2 Schrodinger unitary problem

In [34]:
t0 = 0.
y0 = np.array([[1., 0.], [0., 1.]], dtype=complex)

# takes the hamiltonian and sets up Schrodinger equation for the unitary
de_problem = UnitaryProblem(hamiltonian=qubit_ham_model,
                            y0=y0,
                            t0=t0)


#solver_options = {'scipy_options' : {'atol': 10**-9, 'rtol': 10**-9}}
solver_options = DE_Options(method='scipy-RK45', atol=10**-8, rtol=10**-8)

scipy_solver = Solver(de_problem, method='scipy-RK45', options=solver_options)

In [35]:
scipy_solver.integrate(1./r) # do a pi pulse
scipy_solver.y

array([[-1.96352698e-05-0.00500054j, -2.82320948e-10+0.99998751j],
       [ 2.82320948e-10+0.99998751j, -1.96352698e-05+0.00500054j]])

### 3.3 Lindblad problem

I broke this from recent changes - but it involved setting up the vectorized version of the Lindblad equation for a density matrix from a Hamiltonian and noise operators, then interacting with the solver in the same way. Even though internally the solver is working with the vectorized form (and hence the flattened density matrix), it returns the state to the user in the correct shape.

## 4. Frame handling

In the above simulations, the solver is automatically doing the simulations in the frame of the drift Hamiltonian. We can instead specify a frame manually.

### 4.1 simulate in lab frame

Default is `transformations='auto'`. Setting to `transformations=None` sets no transformations. 

In [36]:
t0 = 0.
y0 = np.array([[1., 0.], [0., 1.]], dtype=complex)

# takes the hamiltonian and sets up Schrodinger equation for the unitary
de_problem = UnitaryProblem(hamiltonian=qubit_ham_model,
                            y0=y0,
                            t0=t0,
                            transformations=None)

#solver_options = {'scipy_options' : {'atol': 10**-9, 'rtol': 10**-9}}
solver_options = DE_Options(method='scipy-RK45', atol=10**-9, rtol=10**-9)

scipy_solver = Solver(de_problem, options=solver_options)

start = time()
scipy_solver.integrate(1. /r)
print(time() - start)
scipy_solver.y

0.4588286876678467


array([[-1.96358997e-05-0.00500056j,  3.93594324e-11+0.99998746j],
       [-3.93597759e-11+0.99998746j, -1.96358997e-05+0.00500056j]])

### 4.2 Simulate allowing automatic transformations 

In [37]:
de_problem = UnitaryProblem(hamiltonian=qubit_ham_model,
                            y0=y0,
                            t0=t0)
#solver_options = {'scipy_options' : {'atol': 10**-9, 'rtol': 10**-9}}
solver_options = DE_Options(method='scipy-RK45', atol=10**-9, rtol=10**-9)

scipy_solver = Solver(de_problem, options=solver_options)

start = time()
scipy_solver.integrate(1. /r)
print(time() - start)
scipy_solver.y

0.2855818271636963


array([[-1.96364748e-05-0.00500056j, -3.70946652e-11+0.9999875j ],
       [ 3.70946652e-11+0.9999875j , -1.96364748e-05+0.00500056j]])

### 4.3 Simulate in frame of drift and do a rotating wave approximation

In [38]:
transformations = {'frame': qubit_ham_model.drift(), 'rwa_freq_cutoff': 2*w}

de_problem = UnitaryProblem(hamiltonian=qubit_ham_model,
                            y0=y0,
                            t0=t0,
                            transformations=transformations)
#solver_options = {'scipy_options' : {'atol': 10**-9, 'rtol': 10**-9}}
solver_options = DE_Options(method='scipy-RK45', atol=10**-9, rtol=10**-9)

scipy_solver = Solver(de_problem, options=solver_options)

start = time()
scipy_solver.integrate(1. /r)
print(time() - start)
scipy_solver.y

0.01359701156616211


array([[ 7.38162517e-11+3.62509162e-26j,  4.91096681e-16+9.99999999e-01j],
       [-4.91096681e-16+9.99999999e-01j,  7.38162517e-11-3.62509162e-26j]])

## 5. Defining a full problem and solving

In the previous sections we defined a BMDE problem with an initial time only, then interacted with a solver object. We can instead define it with a full interval and solve the DE without ever interacting with the the solver object.

In [39]:
de_problem = UnitaryProblem(hamiltonian=qubit_ham_model,
                            y0=y0,
                            interval=[t0, 1./r])

solve(de_problem)

array([[-1.95675357e-05-0.00499976j,  3.47445729e-08+0.99998882j],
       [-3.47445729e-08+0.99998882j, -1.95675357e-05+0.00499976j]])